<a href="https://colab.research.google.com/github/VenkteshV/Question_duplicate_detection/blob/main/Week8/Calculating_sent_emb_cosine_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import spatial

In [ ]:
!pip install --quiet transformers
!pip install --quiet tensorboard
!pip install --quiet  sentence_transformers
import pandas as pd
import numpy as np
import torch
import pandas as pd
from time import time
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import tensorflow_hub as hub # For USE
from sentence_transformers import SentenceTransformer
import os
import tensorflow as tf
import datetime
from tensorboard.plugins import projector


def generate_embeddings(model_name, data_ls):
    if model_name == "USE":
        print("Check 1")
        embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
        print("Check 2")
        embeddings = embed(data_ls)
        print("Check 3")
        embeddings = embeddings.numpy()
        print("Check 4")
        return embeddings

    elif model_name == "all-MiniLM":
        # All-round model tuned for many use-cases. Trained on a large and diverse dataset of over 1 billion training pairs.
        model = SentenceTransformer('all-MiniLM-L12-v2')
        embeddings = model.encode(data_ls)
        return embeddings

    elif model_name == "paraphrase-MiniLM-L6-v2":
        model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        embeddings = model.encode(data_ls)
        return embeddings

    elif model_name == "multi-qa-MiniLM-L6-cos-v1":
        # This model was tuned for semantic search: Given a query/question, if can find relevant passages. It was trained on a large and diverse set of (question, answer) pairs.
        model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
        embeddings = model.encode(data_ls)
        return embeddings

    elif model_name == "multi-qa-mpnet-base-dot-v1":
        # This model was tuned for semantic search: Given a query/question, if can find relevant passages. It was trained on a large and diverse set of (question, answer) pairs.
        model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
        embeddings = model.encode(data_ls)
        return embeddings
    elif model_name == "all-mpnet-base-v2":
        # All-round model tuned for many use-cases. Trained on a large and diverse dataset of over 1 billion training pairs.
        model = SentenceTransformer('all-mpnet-base-v2')
        embeddings = model.encode(data_ls)
        return embeddings

    elif model_name == "paraphrase-albert-small-V2":
        model = SentenceTransformer('paraphrase-albert-small-v2')
        embeddings = model.encode(data_ls)
        return embeddings

    elif model_name == "all-distilroberta-v1":
        model = SentenceTransformer('all-distilroberta-v1')
        embeddings = model.encode(data_ls)
        return embeddings

    else:
        print("sorry")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Code

In [ ]:
df = pd.read_csv("drive/My Drive/Extramarks Project/Dataset versions/JEE-ans-no_num-no_fillBlank-replaced_ofthefollowing-removed_one_letter_sent_also.csv")
df = df.drop("Unnamed: 0", axis = 1)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head()

,question_id,question,syllabus_id,answer,question_type,len,words
0,2073346,how many π bonds are present in ferrocene,700295,a,Subjective Question,41,8
1,2073324,what is covalency of chlorine atom in second e...,698013,a,Subjective Question,58,10
2,2073225,in solid x atoms goes to corner of the cube an...,700078,No of x atoms 8 18 2 12 2,Subjective Question,126,25
3,2073215,calculate value of x y here x and y are the to...,697895,stryde images 2022 04 11 202204110162130016496...,Subjective Question,156,28
4,2073202,in phosphorus acid if x is number of non bondi...,697895,stryde images 2022 04 11 202204119693370016496...,Subjective Question,143,32


In [ ]:
len(df) - df['question_id'].nunique()

0

In [ ]:
questions = df['question'].tolist()

In [ ]:
t =time()
embeds = generate_embeddings("all-mpnet-base-v2", questions)
print("Time taken: " , int((time() - t)%60) ,"s, ", int((time() - t)/60), "m")

In [ ]:
embedding_df = pd.DataFrame(embeds)
path = "drive/MyDrive/Extramarks Project/Dataset versions/Model1-Semantic/Embeddings/output.tsv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  embedding_df.to_csv(f, index=False, sep='\t')

In [ ]:
sentences_df = pd.DataFrame(questions)
path = "drive/MyDrive/Extramarks Project/Dataset versions/Model1-Semantic/Embeddings/metadata.tsv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  sentences_df.to_csv(f, index=False, sep='\t')

In [ ]:
question_ids = df['question_id'].tolist()

In [ ]:
ques_to_embed = dict(zip(question_ids, embeds))

In [ ]:
newDf = pd.read_csv("drive/My Drive/Extramarks Project/Dataset versions/Model1-Semantic/Jacc_sim_ignore_syllabus.csv")
newDf = newDf.drop("Unnamed: 0", axis = 1)

In [ ]:
newDf.head()

In [ ]:
cos_similarities = []
for index, row in newDf.iterrows():
  cos_sim = 1 - spatial.distance.cosine(ques_to_embed[row['question_id1']], ques_to_embed[row['question_id2']])
  cos_similarities.append(cos_sim)

In [ ]:
newDf['cos_sim'] = cos_similarities
newDf.to_csv("drive/My Drive/Extramarks Project/Dataset versions/Model1-Semantic/JaccPlusEmbedCosinSim.csv")